![](image.jpg)

In the realm of global finance, debt is not just a tool for individuals but a crucial instrument for nations striving to bolster their economies. Imagine a country embarking on an ambitious journey to enhance its infrastructure, aiming to provide its citizens with a higher quality of life. This is where the World Bank steps in, offering financial support to developing nations.

In this project, you will delve into a fascinating dataset curated by the World Bank, which encapsulates the debt profiles of various developing countries. Your mission is to uncover insights hidden within this data, answering pivotal questions such as:

- How many distinct countries are represented in this dataset?
- Which country holds the highest amount of debt?
- Which country has the lowest amount of repayments?

To embark on this analytical journey, you will work with the following table:

## `international_debt` table

| Column | Definition | Data Type |
|-|-|-|
| `country_name` | Name of the country | `varchar` |
| `country_code` | Code representing the country | `varchar` |
| `indicator_name` | Description of the debt indicator | `varchar` |
| `indicator_code` | Code representing the debt indicator | `varchar` |
| `debt` | Value of the debt indicator for the given country (in current US dollars) | `float` |

Our task will be is to craft SQL queries that will illuminate the answers to these compelling questions above. Let's dive into the data and uncover the stories it holds, shall we?

First of all, to find insghtful and accurate answers to our questions we have to first have a sense of the data that we have. So, we will first take a look at the records and the fields that we have.

In [ ]:
/* Explore our data and our fileds by taking a sample of 100 records to get an understanding of what we are working with*/
SELECT *
FROM international_debt
LIMIT 100;

,country_name,country_code,indicator_name,indicator_code,debt
0,Afghanistan,AFG,"Disbursements on external debt, long-term (DIS...",DT.DIS.DLXF.CD,7.289445e+07
1,Afghanistan,AFG,"Interest payments on external debt, long-term ...",DT.INT.DLXF.CD,5.323944e+07
2,Afghanistan,AFG,"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD,6.173934e+07
3,Afghanistan,AFG,"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD,4.911473e+07
4,Afghanistan,AFG,"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD,3.990362e+07
...,...,...,...,...,...
95,Armenia,ARM,"PPG, private creditors (INT, current US$)",DT.INT.PRVT.CD,6.595069e+07
96,Armenia,ARM,"Principal repayments on external debt, long-te...",DT.AMT.DLXF.CD,8.456307e+08
97,Armenia,ARM,"Principal repayments on external debt, private...",DT.AMT.DPNG.CD,6.806962e+08
98,Azerbaijan,AZE,"Disbursements on external debt, long-term (DIS...",DT.DIS.DLXF.CD,1.088458e+09


**So, taking a quick look at our data in our table, we can see the following 5 fields in our provided table which are:**
- `country_name`: which contains the countrys' names that are involved in debt
- `country_code`: which containt the codes for each country which act as a unique identifier
- `indicator_name`: You can think about the indicator's name as a name that indicates a specific type of debt
- `indicator_code`: It uniqely identifies the indicator name
- `debt`: The actual number of debt that the country has, relative to each type of debt (`indicator_name`)

With that out of the way, now let's see how many countries do we actually have in our dataset.

In [57]:
/* How many unique (NON-Duplicate) countries available in our dataset*/
SELECT DISTINCT country_name AS country
FROM international_debt
ORDER BY country;

,country
0,Afghanistan
1,Albania
2,Algeria
3,Angola
4,Armenia
...,...
119,"Venezuela, RB"
120,Vietnam
121,"Yemen, Rep."
122,Zambia


In [58]:
/* Let's get the count of all these countries */
SELECT COUNT(DISTINCT country_name) AS distinct_country
FROM international_debt;

,distinct_country
0,124


Great! We now know that we have 124 `DISTINCT` countries in our dataset.

Let's analyze the data a bit more by g**rouping the number of debt, or to be specific, the number of indicator names assigned to each country to see how much type of debt each country has.**

We can do that by grouping the country names and counting the indicator name for each one and assigned it to them.

In [59]:
/* SELECT the country name aliasing it, and count the indicator name of each type of debt to each country and group by each country */
SELECT country_name as country, COUNT(indicator_name) AS sum_of_indicator_names
FROM international_debt
GROUP BY country_name
ORDER BY 2 DESC, 1;

,country,sum_of_indicator_names
0,Albania,25
1,Cameroon,25
2,Dominican Republic,25
3,Ghana,25
4,IDA only,25
...,...,...
119,"Yemen, Rep.",12
120,Guinea-Bissau,11
121,Somalia,11
122,Timor-Leste,10


Great! Seems like the largest possible number of indicator names assigned to any country is `25` and the lowest is `10`.

**But does that mean that the more a country has indicator names assigned to it, the more does it have debt?**

Let's find that out! We can use start of by selecting the country names, the number of indicator names assigned to each country and the sum of all debt issued by each country, grouping by country names of course.

In [60]:
/* Here we have selected each country's name and the total_debt in the main query
   1. We also made two sub quereies, one in the FROM where we made a table that has each country's name
      and and the SUM of all debts of each indicator name assigned to each country all of that is grouped       on to countries and given a name 'total_debts_for_each_country'
   
   2. Then after that we added a WHERE clause that filters with the total_debt that supposed to indicate       the country that has the highest total debt as it returns a single value which indicates 		         285793494734.2 */
SELECT country AS country, ROUND(total_debt) 
FROM(
	SELECT country_name AS country, SUM(debt) AS total_debt
	FROM international_debt 
	GROUP BY country_name) AS total_debts_for_each_country
WHERE total_debt =
	(SELECT MAX(total_debt)
     FROM (SELECT SUM(debt) AS total_debt
		   FROM international_debt 
		   GROUP BY country_name) AS max_query); 

,country,round
0,China,285793494734


In [61]:
/* Let's see how much indicator names does China has */
SELECT COUNT(indicator_name)
FROM international_debt 
WHERE country_name = 'China';

,count
0,24


Looks like the country that has the highest number of debt is China with a `total_debt` of 285793494734.2
and has a sum of 24 `indicator_name`

Now, let's do the same thing but with the opposite. Let's get the minimum debt and identify which country that has the mimimum debt assigned to it. **And see maybe if there is a pattern or a trend between the `indicator_name` and the `total_debt`**

In [62]:
/* Do the same thing but with minimum rather than maximum */
SELECT country AS country, ROUND(total_debt) 
FROM(
	SELECT country_name AS country, SUM(debt) AS total_debt
	FROM international_debt 
	GROUP BY country_name) AS total_debts_for_each_country
WHERE total_debt =
	(SELECT MIN(total_debt)
     FROM (SELECT SUM(debt) AS total_debt
		   FROM international_debt 
		   GROUP BY country_name) AS max_query); 

,country,round
0,Sao Tome and Principe,44798033


In [63]:
/* Let's see how much indicator names does China has */
SELECT COUNT(indicator_name)
FROM international_debt 
WHERE country_name = 'Sao Tome and Principe';

,count
0,12


Sao Tome and Principe has the lowest total debt but has a SUM of 12 `indicator_name`. It has a total debt of 447980033

Great! Now, let's take another look at the `country_name`, the sum of `indicator_name` and the `total_debt` of each country.

In [64]:
SELECT country, num_of_indicators, num_of_total_debt
FROM (SELECT country_name AS country, COUNT(indicator_name) AS num_of_indicators, SUM(debt) AS                   num_of_total_debt
      FROM international_debt
	  GROUP BY country_name) AS total_debt
ORDER BY 3 DESC;

,country,num_of_indicators,num_of_total_debt
0,China,24,2.857935e+11
1,Brazil,24,2.806240e+11
2,South Asia,24,2.476087e+11
3,Least developed countries: UN classification,25,2.128810e+11
4,Russian Federation,21,1.912891e+11
...,...,...,...
119,Turkmenistan,10,9.513563e+07
120,Timor-Leste,10,7.100096e+07
121,Comoros,12,5.756848e+07
122,Tonga,15,5.520930e+07


That is weird the Phillippines has a less total number of debt than let's say Peru but it is ordered as higher, meaning having more debt than it so let's do a comparision to check our logical code.

In [65]:
SELECT country_name, SUM(debt) AS total_debt
FROM international_debt 
GROUP BY country_name
HAVING country_name = 'Peru' OR country_name = 'Philippines'
ORDER BY total_debt DESC;

,country_name,total_debt
0,Philippines,2.515171e+10
1,Peru,2.409178e+10


So the problem was not in the logical sequence of code. It was in the visual appearance of the values. Specifically, the rounding of the values to the nearest deciaml value. We can adjust this by rounding it correclty.

In [66]:
SELECT country, num_of_indicators, ROUND(num_of_total_debt)
FROM (SELECT country_name AS country, COUNT(indicator_name) AS num_of_indicators, SUM(debt) AS                   num_of_total_debt
      FROM international_debt
	  GROUP BY country_name) AS total_debt
ORDER BY 3 DESC, 2 DESC, 1;

,country,num_of_indicators,round
0,China,24,285793494734
1,Brazil,24,280623966141
2,South Asia,24,247608723991
3,Least developed countries: UN classification,25,212880992792
4,Russian Federation,21,191289057259
...,...,...,...
119,Turkmenistan,10,95135629
120,Timor-Leste,10,71000959
121,Comoros,12,57568483
122,Tonga,15,55209305


Here, that is better!

Last question we need to address is that we need to identify the country that has the lowest amount of principal repayments indicated by code "_DT.AMT.DLXF.CD_"

In [67]:
SELECT country_name
FROM international_debt 
WHERE indicator_code = 'DT.AMT.DLXF.CD' AND debt = (SELECT MIN(debt) 
												    FROM international_debt 
												    WHERE indicator_code = 'DT.AMT.DLXF.CD');

,country_name
0,Timor-Leste


We have answered all our questions.
1. What is the number of distinct countries present in the database?
**124**

2. What country has the highest amount of debt? **China**

3. What country has the lowest amount of principal repayments? **Timor-Leste**